Основано на [тьюториале](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb).

In [1]:
%pip install Partial State
%pip install --upgrade accelerate
%pip install --upgrade transformers
%pip install datasets transformers seqeval corus razdel -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


В этом блокноте мы дообучаем модель на задаче классификации отдельных слов, а именно, распознавание именованных сущностей (aka named entity recognition, aka NER). Мы возьмём датасет медицинских сущностей, но в целом пайплайн подходит для любой задачи на выделение сущностей в тексте.

Для скорости мы возьмём маленький BERT для русского языка [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny2); если взять другую, более крупную BERT-подобную модель, качество NER может быть выше, но и время обучения и работы будет дольше


In [2]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 16

## Загрузим датасет

Для обучения мы возьмём [Russian Drug Reaction Corpus](https://github.com/cimm-kzn/RuDReC): размеченный корпус русскоязычных отзывов на лекарства.

Загрузим мы его библиотекой corus, потому что это удобно

In [3]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2024-06-17 21:04:54--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [переход]
--2024-06-17 21:04:55--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1773014 (1,7M) [text/plain]
Сохранение в: «rudrec_annotated.json.1»

rudrec_annotated.js 100%[===================>]   1,69M  6,70MB/s    за 0,3s    

2024-06-17 21:04:55 (6,70 MB/s) - «rudrec_annotated.json.1» сохра

In [4]:
from corus import load_rudrec
from datasets import load_dataset, load_metric
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


/Users/andrey/projects/otus/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Пример документа:

In [5]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

Посмотрим, какие сущности есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Название/бренд препарата или его ингредиенты/активные соединения.
* **DRUGCLASS** Класса лекарства (противовоспалительное,
сердечно-сосудистое и т. п.).
* **DRUGFORM** Форма препарата (таблетки, сироп и т. п.).
* **DI** Любые симптомы, которые могли быть показанием к назначению препарата.
* **ADR** Побочные эффекты от приема препарата.
* **FINDING** Любые побочные эффекты, напрямую не связанные с приемом препарата или исходной болезнью, которые наблюдались у пациента.


In [6]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [7]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Напишем функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд.

In [8]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [9]:
print(extract_labels(drugs[0])['tags'])

['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [10]:
print(extract_labels(drugs[0])['tokens'])

['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?']


In [11]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

Пример данных

In [12]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
712,"[Вакцинацию, 9, летняя, дочка, перенесла, превосходно, ,, никаких, симптомов, гриппозного, состояния, .]","[O, O, O, O, O, O, O, O, O, O, O, O]"
3642,"[Однажды, у, ребенка, поднялась, высокая, температура, и, чтобы, ее, сбить, я, решила, дать, парацетамол, для, детей, в, суспензит, .]","[O, O, O, O, B-DI, I-DI, O, O, O, O, O, O, O, B-Drugname, O, O, O, B-Drugform, O]"
3215,"[О, препарате, Полиоксидоний, узнала, не, так, давно, .]","[O, O, B-Drugname, O, O, O, O, O]"


Соберём все виды меток в список.

In [13]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Сложим наши данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [14]:
from datasets import Dataset, DatasetDict

In [15]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

## Предобработка данных

Прежде чем мы сможем подать тексты на вход модели, их нужно предобработать. Сделать это можно с помощью токенизатора `Transformers`, который токенизирует входные данные и преобразует их в формат, ожидаемый моделью, а также генерирует другие входные данные, которые которые необходимы для работы модели.

Для этого воспользуемся классов `AutoTokenizer.from_pretrained`.

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Вызов токенизатора осуществляется в одну строчку:

In [17]:
tokenizer("Привет, это одно предложение!")

{'input_ids': [2, 51343, 16, 2389, 13305, 24889, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}



Если, как в нашем случае, входные данные уже были разделены на слова, нужно передать токенизатору список слов с аргументом `is_split_into_words=True`.

In [18]:
tokenizer(["Привет", ",", "это", "одно", "предложение", ",", "разбитое", "на", "слова", "."], is_split_into_words=True)

{'input_ids': [2, 51343, 16, 2389, 13305, 24889, 16, 52423, 626, 548, 7851, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Вернемся к исходному формату.

In [19]:
tokenized_input = tokenizer("Привет, это одно предложение!", is_split_into_words=False)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Привет', ',', 'это', 'одно', 'предложение', '!', '[SEP]']


In [20]:
tokenized_input = tokenizer(["Привет", ",", "это", "одно", "предложение", ",", "разбитое", "на", "слова", "."], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Привет', ',', 'это', 'одно', 'предложение', ',', 'разбито', '##е', 'на', 'слова', '.', '[SEP]']


In [21]:
tokenized_input["input_ids"]

[2, 51343, 16, 2389, 13305, 24889, 16, 52423, 626, 548, 7851, 18, 3]

Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [22]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, None]


In [23]:
example = ner_train[5]
print(example["tokens"])


['Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.']


In [24]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

13 13


In [25]:
aligned_labels

[-100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100]

In [26]:
tokenized_input["input_ids"]

[2, 51343, 16, 2389, 13305, 24889, 16, 52423, 626, 548, 7851, 18, 3]

Сопоставим специальным токенам лейблы -100 (этот индекс игнорируется PyTorch), а всем остальным токенам поставим в соотвествие лейблы тех слов, к которым они относятся. Другая стратегия - сопоставить лейбл только для первого токена каждого слова, а остальным присвоить метку -100. Как показывает практика, вторая стратегия обычно работает лучше. Реализуем обе стратегии, а переключаться между ними будем с помощью флага `label_all_tokens`.

In [27]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Функция способна работать с одним или несколькими примерами. В случае нескольких примеров токенизатор вернет список списков для каждого ключа:

In [28]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 1041, 37038, 33265, 19106, 40305, 22018, 548, 22276, 320, 21538, 16, 47886, 548, 59614, 11137, 626, 56606, 700, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 1, -100, 7, 7, 7, 7, 0, 0, 0, 0, -100, -100, 0, -100, 0, -100]]}

In [29]:
ner_data['train'][22:23]

{'tokens': [['На',
   'следующее',
   'утро',
   'появились',
   'высыпания',
   'на',
   'лице',
   'и',
   'руках',
   ',',
   'похожие',
   'на',
   'комариные',
   'укусы',
   '.']],
 'tags': [['O',
   'O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']]}

Применим данную функйию предобработки ко всему датасету.

In [30]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 962/962 [00:00<00:00, 24901.08 examples/s]


## Fine-tuning the model

В данном случае мы решаем дазачу Sequence Labeling, поэтому для дообучения модели воспользуемся классем `AutoModelForTokenClassification`.

In [31]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [32]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Зададим параметры обучения с помощью класса [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments).

In [33]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

/Users/andrey/projects/otus/.venv/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Затем нам понадобится data collator для корректной обработки данных по батчам.

In [34]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Для оценки качества будем использовать метрику `precision` из библиотеки [`seqeval`](https://github.com/chakki-works/seqeval), которая считает данную метрику без учета "пустых" слов, не являющимися именованными сущностями.

In [36]:
metric = load_metric("seqeval")

In [37]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'DI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Drugform': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Для вычисления метрики потребуется еще несколько шагов постпроцессинга:
- для каждого токена выбрать метку, соотвествующую наибольшей вероятности
- получить список предсказаний
- игнорировать токены с меткой -100


In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Наконец воспользуемся классем `Trainer` для тренировки модели.



In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.evaluate()

100%|██████████| 61/61 [00:04<00:00, 13.47it/s]


{'eval_loss': 2.4529662132263184,
 'eval_precision': 0.00684931506849315,
 'eval_recall': 0.080338266384778,
 'eval_f1': 0.012622487958810828,
 'eval_accuracy': 0.1116938760434365,
 'eval_runtime': 6.7842,
 'eval_samples_per_second': 141.801,
 'eval_steps_per_second': 8.992}

Дообучим модель, используя метод `train`.

In [41]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [42]:
trainer.train()

                                                  
  5%|▌         | 244/4820 [00:29<16:33,  4.61it/s]

{'eval_loss': 0.3201746344566345, 'eval_precision': 0.49782923299565845, 'eval_recall': 0.36363636363636365, 'eval_f1': 0.4202810018326206, 'eval_accuracy': 0.924281598581665, 'eval_runtime': 0.7979, 'eval_samples_per_second': 1205.671, 'eval_steps_per_second': 76.451, 'epoch': 1.0}


                                                  
 10%|█         | 484/4820 [00:43<15:19,  4.71it/s]

{'eval_loss': 0.2618018686771393, 'eval_precision': 0.4778225806451613, 'eval_recall': 0.5010570824524313, 'eval_f1': 0.4891640866873065, 'eval_accuracy': 0.9308561719731108, 'eval_runtime': 0.6911, 'eval_samples_per_second': 1391.891, 'eval_steps_per_second': 88.259, 'epoch': 2.0}


 10%|█         | 503/4820 [00:44<04:27, 16.12it/s]

...

                                                   
 95%|█████████▌| 4583/4820 [04:40<00:37,  6.32it/s]

{'eval_loss': 0.17399899661540985, 'eval_precision': 0.6368613138686131, 'eval_recall': 0.7378435517970402, 'eval_f1': 0.683643486777669, 'eval_accuracy': 0.9553076752603974, 'eval_runtime': 0.7339, 'eval_samples_per_second': 1310.729, 'eval_steps_per_second': 83.113, 'epoch': 19.0}


                                                   
100%|██████████| 4820/4820 [04:53<00:00, 16.41it/s]

{'eval_loss': 0.17357298731803894, 'eval_precision': 0.6438482886216467, 'eval_recall': 0.7357293868921776, 'eval_f1': 0.6867291563887519, 'eval_accuracy': 0.9556770333160967, 'eval_runtime': 0.7376, 'eval_samples_per_second': 1304.234, 'eval_steps_per_second': 82.701, 'epoch': 20.0}
{'train_runtime': 293.7024, 'train_samples_per_second': 261.966, 'train_steps_per_second': 16.411, 'train_loss': 0.14651404455984282, 'epoch': 20.0}


TrainOutput(global_step=4820, training_loss=0.14651404455984282, metrics={'train_runtime': 293.7024, 'train_samples_per_second': 261.966, 'train_steps_per_second': 16.411, 'total_flos': 49424108009730.0, 'train_loss': 0.14651404455984282, 'epoch': 20.0})

In [43]:
trainer.evaluate()

100%|██████████| 61/61 [00:00<00:00, 93.99it/s] 


{'eval_loss': 0.17357298731803894,
 'eval_precision': 0.6438482886216467,
 'eval_recall': 0.7357293868921776,
 'eval_f1': 0.6867291563887519,
 'eval_accuracy': 0.9556770333160967,
 'eval_runtime': 0.7975,
 'eval_samples_per_second': 1206.204,
 'eval_steps_per_second': 76.485,
 'epoch': 20.0}

Посчитаем precision/recall/f1 для каждой категории.

In [44]:
p = trainer.predict(tokenized_datasets["test"])


predictions, labels, inputs = p.predictions, p.label_ids, tokenized_datasets["test"]['input_ids']
predictions = np.argmax(p.predictions, axis=2)

# send only the first token of each word to the evaluation
true_predictions = []
true_labels = []
for prediction, label, tokens in zip(predictions, labels, inputs):
    true_predictions.append([])
    true_labels.append([])
    for (p, l, t) in zip(prediction, label, tokens):
        if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
            true_predictions[-1].append(label_list[p])
            true_labels[-1].append(label_list[l])

results = metric.compute(predictions=true_predictions, references=true_labels)
results

100%|██████████| 61/61 [00:00<00:00, 89.58it/s] 


{'ADR': {'precision': 0.34299516908212563,
  'recall': 0.47651006711409394,
  'f1': 0.398876404494382,
  'number': 149},
 'DI': {'precision': 0.5013927576601671,
  'recall': 0.6666666666666666,
  'f1': 0.5723370429252782,
  'number': 270},
 'Drugclass': {'precision': 0.8095238095238095,
  'recall': 0.9315068493150684,
  'f1': 0.8662420382165604,
  'number': 73},
 'Drugform': {'precision': 0.873015873015873,
  'recall': 0.9593023255813954,
  'f1': 0.9141274238227148,
  'number': 172},
 'Drugname': {'precision': 0.8828451882845189,
  'recall': 0.9336283185840708,
  'f1': 0.9075268817204302,
  'number': 226},
 'Finding': {'precision': 0.3333333333333333,
  'recall': 0.017857142857142856,
  'f1': 0.03389830508474576,
  'number': 56},
 'overall_precision': 0.6438482886216467,
 'overall_recall': 0.7357293868921776,
 'overall_f1': 0.6867291563887519,
 'overall_accuracy': 0.9556770333160967}

In [45]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [46]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,12034,17,54,7,16,12,0,23,31,0,0,1,0
B-ADR,27,87,28,1,0,0,2,4,0,0,0,0,0
B-DI,37,17,198,5,2,2,0,2,7,0,0,0,0
B-Drugclass,4,0,0,68,1,0,0,0,0,0,0,0,0
B-Drugform,6,0,0,0,166,0,0,0,0,0,0,0,0
B-Drugname,4,0,1,1,1,219,0,0,0,0,0,0,0
B-Finding,10,24,20,0,0,1,1,0,0,0,0,0,0
I-ADR,52,9,4,0,0,0,0,76,28,0,0,0,0
I-DI,61,6,14,0,1,0,0,20,74,0,0,0,0
I-Drugclass,0,0,0,1,0,0,0,0,0,0,0,0,0


In [47]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Применение модели

In [48]:
import torch

In [49]:

text = ' '.join(ner_test[4]['tokens'])  # 458 is example of a difficult text
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [50]:
tokens = tokenizer(text, return_tensors='pt').to(model.device)

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 21, 13])

In [51]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
Охо             O         
##тно           O         
применя         O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
насмор          B-DI      
##ком           O         
,               O         
что             O         
в               O         
моем            O         
случае          O         
явление         O         
очень           O         
частое          O         
.               O         
[SEP]           O         


Более простое применение модели: пайплайн от huggingface

In [52]:
from transformers import pipeline

In [53]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [54]:
print(text)
print(pipe(text))

Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .
[{'entity_group': 'DI', 'score': 0.47364157, 'word': 'насморком', 'start': 33, 'end': 42}]
